In [ ]:
## Importamos librerias para el EDA
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns

## Leemos el dataset en un dataframe
bank = pd.read_csv("bank_subscription.csv")

In [ ]:
bank = bank.drop("Unnamed: 0",axis=1)

### NaNs
Teniendo en cuenta que tenemos un 76% de las filas del dataset con al menos un valor faltante, no podemos eliminar los mismos indiscriminadamente, por lo que haremos algunas asunciones.

- Credit: Todos aquellos valores que sean faltantes seran transformados a "no". La lógica es que si tuvieran un credito en el banco esto se vería reflejado en el dataset
- Balances (euros): Todos aquellos valores que sean faltantes seran transformados a 0. La lógica es que si tuvieran balance en el banco esto se vería reflejado en el dataset
- Housing/personal loan: Todos aquellos valores que sean faltantes seran transformados a "no". La lógica es que si tuvieran un prestamo en el banco esto se vería reflejado en el dataset
- Pdays: Todos aquellos valores que sean faltantes seran transformados a -1. La lógica es que si hubiese habido un contacto se vería reflejado en el dataset.
- Last contact duration: Todos aquellos valores que sean faltantes seran transformados a 0. La lógica es que si hubiese habido un contacto se vería reflejado en el dataset.

Todas las variables mas ajenas al banco como edad, trabajo, estado civil y educación, no las rellenaremos porque significaria introducir mucho ruido en el dataset. Esperamos que con estos reemplazos lleguemos a un numero aceptable de NaNs para eliminar las filas que contengan valores nulos


In [ ]:
## Creamos un diccionario con los valores a reemplazar y utilizamos el metodo .fillna() para remplazar valores nulos
values = {"Pdays": -1, "Credit": "no", "Balance (euros)": 0, "Housing Loan": "no", "Personal Loan": "no", "Last Contact Duration": 0}
bank = bank.fillna(value=values)

In [ ]:
## Volvemos a revisar cuantas filas tienen valores nulos para analizar si luego de los cambios podemos eliminarlos
filas_nan = bank.isnull().any(axis=1).sum()
print(filas_nan)
porcentaje_nan = filas_nan/bank.shape[0]*100
print(porcentaje_nan)

16830
37.2254539824379


Habiendo disminuido el porcentaje de filas con algún valor nulo de 76% a 37%, contamos con un valor aceptable para poder eliminar aquellas filas con valores NaN.

In [ ]:
bank = bank.dropna()
print(bank.shape)

(28381, 17)


In [ ]:
## Reemplazamos los valores de 2 y 1 en la variable Subscription para que queden en 1 y 0
bank['Subscription'] = bank['Subscription'].replace(1,0)
bank['Subscription'] = bank['Subscription'].replace(2,1)

In [ ]:
# Que variables son categoricas?
categorical = [var for var in bank.columns if bank[var].dtype=='O']

print('Hay {} Variables categoricas\n'.format(len(categorical)))

print('Las variables categoricas son :', categorical)

## Scaling


In [ ]:
# Que variables son numericas?
numerical = [var for var in bank.columns if bank[var].dtype!='O']

print('Hay {} variables numericas \n'.format(len(numerical)))

print('The numerical variables are :', numerical)

In [ ]:
# Separamos las features de la variable objetivo
X = bank.drop(['Subscription'], axis=1)

y = bank['Subscription']

In [ ]:
# Seleccionamos las variables numericas nuevamente sin la columna Subscription
numerical_x = [var for var in X.columns if X[var].dtype!='O']

In [ ]:
# Separamos en test y train
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

X_train.shape, X_test.shape


In [ ]:
# Dividimos las variables categoricas de las numericas para poder realizar scaling y PCA

X_train_cat = X_train[categorical]
X_train_num = X_train[numerical_x]
X_test_cat = X_test[categorical]
X_test_num = X_test[numerical_x]

In [ ]:
# Vamos a tener que convertir esta variables categoricas en dummies para que puedan ingresar al modelo
X_train_cat = pd.get_dummies(data=X_train_cat, columns=categorical)
X_test_cat = pd.get_dummies(data=X_test_cat, columns=categorical)